
# Final Project.

## Game: "Gladiator"


Simple RPG based game about gladiator fights in Colliseum. Combat resolves turn by turn.

### 1. Game Description.

#### 1.1 Gameplay.

The gameplay is following:  
Player can create character with different attributes such as <b>strength, dexterity and combat</b> by assigning free skill points to those attributes.  
Than game screen appears:  
<li>Player has limited space for movement, some for monster (opponent). 
<li>Player can move character in Up, Down, Right and Left direction.  
<li>If player is close to an item - he can pick an item.  
<li>If player is close to the monster, combat mode starts automaticaly.
<li>Player and monster have <b>Health Points (HP)</b>. 
<li>To win in combat <b>HP</b> of opponent must be reduced to 0.  
<li>If players <b>HP</b> are reduced to 0, player looses and game starts once again.  
<li>If player kills a monster <b>kills</b> counter is increased by 1.  
<li> Difficulty level is progressive and depends on number of <b>kills</b>

    
Goal: Kill as many as you can!


#### 1.2 Game Mechanics.  
  
The mechanics is based on simplified adoptation of Dungeons & Dragons tabletop role playing game.  
At the heart of the game mechanics lies rolls of D20 (20 sided) dice plus modifiers.  
  
Key rules:  
<li>Each creature has <b>HP</b>. 

<li>Players <b>HP</b> is calculated as follows: <b> 10 </b> + <b> Strength </b> Attribute. 

<li>There are <b>Medkits</b> in game. Player can pick up <b>Medkits</b> and use anytime during game. <b>Medkits</b> gives <b>+ 5 HP</b>

<li>Each creature has <b> Armor Class (AC)</b>  which is calculated as follows: <b>+ Dexterity </b> Attribute.  

<li>Each creature has <b> Attack Bonus</b> which is calculated as follows: <b>+ Combat </b> Attribute.  

<li>For attack to be successful, <b> D20 + Attack Bonus + Attack Bonus Modifier</b> should be greater than <b> AC + AC Modifier</b> of defender.  

<li>If Attack was successful than damage is calculated as follows: <b>for short attack: D3 + Strength // 2 </b>  
<b> for long attack: D6 + Strength // 2 </b>  

<li>Player can <b>defend</b>, defence <b>increases AC by 3</b> for one round.  

<li><b> short attack</b> has no <b>Attack Modifiers</b> and takes one round.  

<li><b>long attack</b> has <b> Attack Modifier = +4</b> and takes 2 rounds.

<li>Monsters are generated with random <b>HP</b>, <b>AC</b> ,<b> Attack Bonus</b> and <b> Damage</b> within a difficulty level range. Range depends on number of kills.  

<li>Same rules applies to monster attack.  

<li>If Monster attack is succesful, constant damage is dealed to a player.




### 2. Game implementation.  

Game is implemented with the use of ipycanvas module for graphics.  

First we import modules:

In [1]:
from ipycanvas import Canvas
from random import randint
from time import sleep

#### 2.1 Functions for graphics.

First function fills the rectangular game screen:

In [2]:
def fill_background():
    global screen_width, screen_hight
    canvas.fill_style = "black"
    canvas.fill_rect(0, 0, screen_width, screen_hight)
    return

This function draws button for GUI.  
Inputs are: starting X and Y coordinates, width, hight, text which will be displayed, font, and status of button.  

If action could be performed, button will be active and highlighted. 

In [3]:
def button(x, y, width, hight, text, text_offset_x=40, text_offset_y=-5, font='24px Sans-serif', active=False):
    canvas.stroke_style = '#50F40C'
    canvas.fill_style = "#50F40C"
    canvas.line_width = 3
    if active == False:
        canvas.stroke_rect(x, y, width, hight)
    else:
        canvas.fill_style = "#F42919"
        canvas.fill_rect(x, y, width, hight)
        canvas.stroke_rect(x, y, width, hight)
        canvas.fill_style = "#50F40C"
    canvas.font = font
    canvas.text_align = 'start'
    canvas.fill_text(text, x + text_offset_x, y + hight - text_offset_y)
    return

This function draws player character.  
Inputs:  starting X and Y coordinates and color.

In [4]:
def draw_player(x, y, color):
    canvas.fill_style = color
    canvas.fill_rect(x, y, 10)
    canvas.fill_rect(x - 10, y + 10, 30, 10)
    canvas.fill_rect(x, y + 20, 10)
    canvas.fill_rect(x - 10, y + 30, 10)
    canvas.fill_rect(x + 10, y + 30, 10)
    return

This function draws main gamescreen and all GUI buttons.  

Inputs: global variables (status of buttons and status of combat mode)  

P.S. Combat buttons are active only in combat mode.

In [5]:
def draw_game_screen():
    global up_btn, r_btn, l_btn, dn_btn, pu_btn, combat_mode
    
    canvas.stroke_style = '#50F40C'
    canvas.line_width = 3
    canvas.stroke_rect(100, 100, screen_width - 200, screen_hight - 200)
    canvas.stroke_rect(130, 130, screen_width - 260, screen_hight - 260)
    canvas.stroke_line(100, 100, 130, 130)
    canvas.stroke_line(100, screen_hight - 100, 130, screen_hight - 130)
    canvas.stroke_line(screen_width - 100, 100, screen_width - 130, 130)
    canvas.stroke_line(screen_width - 100, screen_hight - 100, screen_width - 130, screen_hight - 130)
    
    font = 'bold 16px Sans-serif'
    button(50, screen_hight - 80, 120, 30, "pick item", text_offset_x = 8, text_offset_y = 8, \
           font=font, active = pu_btn )
    button(50, screen_hight - 40, 120, 30, "defend", text_offset_x = 8, text_offset_y = 8, \
           font=font, active = combat_mode )
    button(200, screen_hight - 80, 120, 30, "short attack", text_offset_x = 8, text_offset_y = 8, \
           font=font, active = combat_mode)
    button(200, screen_hight - 40, 120, 30, "long attack", text_offset_x = 8, text_offset_y = 8, \
           font=font, active = combat_mode)
    button(350, screen_hight - 80, 120, 30, "use medkit", text_offset_x = 8, text_offset_y = 8, \
           font=font, active =  medkits > 0)
    button(350, screen_hight - 40, 120, 30, "run away", text_offset_x = 8, text_offset_y = 8, \
           font=font, active = combat_mode)
    
    font = 'bold 24px Sans-serif'
    button(600, screen_hight - 80, 30, 30, "U", text_offset_x = 7, text_offset_y = 8, font=font, \
           active = up_btn and (not combat_mode))
    button(600, screen_hight - 40, 30, 30, "D", text_offset_x = 7, text_offset_y = 8, font=font, \
           active = dn_btn and (not combat_mode))
    button(560, screen_hight - 60, 30, 30, "L", text_offset_x = 7, text_offset_y = 8, font=font, \
           active = l_btn and (not combat_mode))
    button(640, screen_hight - 60, 30, 30, "R", text_offset_x = 7, text_offset_y = 8, font=font, \
           active = r_btn and (not combat_mode))
    return

This function draws player character.  

Inputs: starting X and Y coordinates and color.

In [6]:
def draw_medkit(x, y):
    canvas.stroke_style = '#50F40C'
    canvas.stroke_line(x, y, x + 20, y)
    canvas.stroke_line(x, y, x , y + 20)
    canvas.stroke_line(x, y + 20, x + 20, y + 20)
    canvas.stroke_line(x + 20, y, x + 20, y + 20)
    canvas.stroke_line(x + 10, y, x + 10, y + 20)
    canvas.stroke_line(x, y + 10, x + 20, y + 10)
    canvas.font = 'bold 18px Courier New'
    canvas.fill_style = "#50F40C"
    canvas.fill_text("medkit", x - 12,y -5)
    return


This function displays game information on top of the screen and results of resolving combat rounds.  

Inputs: global variables (Strength, Dexterity, Combat, ammount of medkits in inventory, HP, Attack Bonus, Attack Modifier, AC Bonus, AC Modifier, text of combat rounds results, monster HP, ammount of Kills)  


In [7]:
def combat_promt():
    global strg, dext, comb, medkits, combat_mode, hp, attack_bonus, \
    attack_modifier, ac_bonus, ac_modifier, promt1, cmbt_promt2, m_hp, kills
    
    canvas.fill_style = "black"
    canvas.fill_rect(0, 0, screen_width, 100)
    canvas.font = '12px Sans-serif'
    canvas.fill_style = "#50F40C"
    canvas.text_align = 'start'
    canvas.fill_text("Strength: "  + str(strg), 10, 20)
    canvas.fill_text("Dexterity: "  + str(dext), 10, 40)
    canvas.fill_text("Combat: "  + str(comb), 10, 60)
    canvas.fill_text("HP: "  + str(hp), 100, 20)
    temp = attack_bonus + attack_modifier
    canvas.fill_text("Attack Bonus: "  + str(temp), 100, 60)
    temp = ac_bonus + ac_modifier
    canvas.fill_text("Armor Bonus: "  + str(temp), 100, 40)
    canvas.fill_text("Iventory: ", 200, 20)
    canvas.fill_text("Medkits: " + str(medkits), 200, 40)
    canvas.fill_text("monster HP: " + str(m_hp), 500, 20)
    canvas.fill_text("Kills: " + str(kills), 500, 40)
    if combat_mode == True:
        canvas.fill_style = "red"
        canvas.font = 'bold 16px Sans-serif'
        canvas.text_align = 'start'
        canvas.fill_text("COMBAT MODE!", 300, 40)
        canvas.font = '13px Sans-serif'
        canvas.fill_text(promt1, 300, 60)
        canvas.fill_text(promt2, 300, 80)
    return

#### 2.2 Handling Mouse Events.  

This function is handling <b>mouse down</b> events.  

It checks which button was pressed if any and calls button action function.

Inputs: Mouse X coordinates, Mouse Y coordinates.  

global variables (Strength, Dexterity, Combat, ammount of medkits in inventory, HP, Attack Bonus, Attack Modifier, AC Bonus, AC Modifier, text of combat rounds results, monster HP, ammount of Kills, screen dimensions, game status, player existance flag, monster existance flag)

In [8]:
def handle_mouse_down(xpos, ypos):
    global screen_flag, strg, dext, comb, pl_x, pl_y, m_x, m_y, med_x, med_y, \
    battle_screen_x_min, battle_screen_x_max, battle_screen_y_min, battle_screen_x_max, \
    up_btn, r_btn, l_btn, dn_btn, medkit_exists, pu_btn, combat_mode, monster_exists, \
    ac_modifier,promt1, promt2, hp,medkits
    
    
    
    if screen_flag == "player_creation":
        if xpos >= 150 and xpos <= 180 and ypos >= 60 and ypos <= 90:
            strg += 1
            player_creation()
            
        if xpos >= 200 and xpos <= 230 and ypos >= 60 and ypos <= 90:
            if strg > 0:
                strg -= 1
                player_creation()
        
        if xpos >= 150 and xpos <= 180 and ypos >= 100 and ypos <= 130:
            dext += 1
            player_creation()
            
        if xpos >= 200 and xpos <= 230 and ypos >= 100 and ypos <= 130:
            if dext > 0:
                dext -= 1
                player_creation()
        
        if xpos >= 150 and xpos <= 180 and ypos >= 140 and ypos <= 170:
            comb += 1
            player_creation()
            
        if xpos >= 200 and xpos <= 230 and ypos >= 140 and ypos <= 170:
            if comb > 0:
                comb -= 1
                player_creation()
    
    elif screen_flag == "game":
        if abs(med_x - pl_x) < 25 and abs(med_y - pl_y) < 25:
            pu_btn = True
        else:
            pu_btn = False

        if abs(pl_x - m_x) < 25 and abs(pl_y - m_y) < 25:
            combat_mode = True
            game()
        else:
            combat_mode = False 
        
        if xpos >= 350 and xpos <= 470 and ypos >= screen_hight - 80 and ypos <= screen_hight - 50:
                use_medkit()
                game()
        
        if combat_mode != True:
            if xpos >= 600 and xpos <= 630 and ypos >= screen_hight - 80 and ypos <= screen_hight - 50:
                if pl_y > battle_screen_y_min: 
                    pl_y -= 20
                    up_btn = True
                    dn_btn = True
                    game()
                else:
                    up_btn = False
                    game()

            if xpos >= 600 and xpos <= 630 and ypos >= screen_hight - 40 and ypos <= screen_hight - 10:
                if pl_y < battle_screen_y_max - 50:
                    pl_y += 20
                    dn_btn = True
                    up_btn = True
                    game()
                else:
                    dn_btn = False
                    game()

            if xpos >= 560 and xpos <= 590 and ypos >= screen_hight - 60 and ypos <= screen_hight - 30:
                if pl_x > battle_screen_x_min + 20:
                    pl_x -= 20
                    l_btn = True
                    r_btn = True
                    game()
                else:
                    l_btn = False
                    game()

            if xpos >= 640 and xpos <= 670 and ypos >= screen_hight - 60 and ypos <= screen_hight - 30:
                if pl_x < battle_screen_x_max - 30:
                    pl_x += 20
                    r_btn = True
                    l_btn = True
                    game()
                else:
                    r_btn = False
                    game()
        
        if xpos >= 50 and xpos <= 170 and ypos >= screen_hight - 80 and ypos <= screen_hight - 50:
            if abs(med_x - pl_x) < 25 and (med_y - pl_y) < 25:
                pick_item()
                medkit_exists = False
                game()
        
        if combat_mode == True:    
            if xpos >= 50 and xpos <= 170 and ypos >= screen_hight - 40 and ypos <= screen_hight - 10:
                defend()
                game()
            
            if xpos >= 200 and xpos <= 320 and ypos >= screen_hight - 80 and ypos <= screen_hight - 50:
                attack("short")
                game()
            
            if xpos >= 200 and xpos <= 320 and ypos >= screen_hight - 40 and ypos <= screen_hight - 10:
                attack("long")
                game()
            
            
            if xpos >= 350 and xpos <= 470 and ypos >= screen_hight - 40 and ypos <= screen_hight - 10:
                combat_mode = False
                monster_exists = False
                promt1 = ""
                promt2 = ""
                game()
        
    return


#### 2.3 Game Mechanics Functions.  

This function is displaying player creation screen until all attributes are spent.  Total ammount of skill point is 7.  

It checks which button was pressed if any and calls button action function.

Inputs: Mouse X coordinates, Mouse Y coordinates.  

global variables (Strength, Dexterity, Combat, HP, Attack Bonus, screen flag)

In [9]:
def player_creation():
    global screen_flag, strg, dext, comb, hp, attack_bonus, ac_bonus 
   
    n = 7 - strg - dext - comb
       
    screen_flag = "player_creation"
    fill_background()

    canvas.fill_style = "#50F40C"
    canvas.font = '24px Sans-serif'
    canvas.text_align = 'start'
    canvas.fill_text("Spend "  + str(n) + " point on skills:", 150,20)
    canvas.fill_text("Strength: " + str(strg), 10,80)
    button(150,60,30,30, "+",text_offset_x = 8,text_offset_y = 8)
    button(200,60,30,30, "-",text_offset_x = 10,text_offset_y = 8)
    canvas.fill_text("Dexterity: " + str(dext), 10,120)
    button(150,100,30,30, "+",text_offset_x = 8,text_offset_y = 8)
    button(200,100,30,30, "-",text_offset_x = 10,text_offset_y = 8)
    canvas.fill_text("Combat: " + str(comb), 10,160)
    button(150,140,30,30, "+",text_offset_x = 8,text_offset_y = 8)
    button(200,140,30,30, "-",text_offset_x = 10,text_offset_y = 8)
    
    if n == 0:
        hp += strg
        attack_bonus += comb
        ac_bonus += dext
        screen_flag = "game"
        game()
    return

This function generates X and Y coordinates such that generated coordinates are > 50 pixels away from inputed ones.  
Inputs: X, Y  
Outputs: X coordinate, Y coordinate

In [10]:
def generate_coordinates(x, y):
    global screen_width, screen_hight
    
    x_obj, y_obj = randint(150, screen_width - 170),randint(150, screen_hight - 170)
    while abs(x_obj - x) <= 50 and abs(y_obj - y) <= 50:
        x_obj, y_obj = randint(150, screen_width - 170),randint(150, screen_hight - 170)
    return x_obj, y_obj

This function adds 1 medkit to inventory.  
Inputs: global variable (medkits)


In [11]:
def pick_item():
    global medkits
    medkits += 1
    return

This function uses 1 medkit from inventory and adds +5 HP. Than calls game function to update screen.  
Inputs: global variable (medkits, HP)

In [12]:
def use_medkit():
    global medkits, hp
    if medkits > 0:
        medkits -= 1
        hp += 5
    game()
    return


This function is resolving <b> defend</b> button.   

It icreases AC modifier, than calls monster attack function to resolve monster turn.  
Than updates player HP and check if player is alive.  
If player dies game starts from begining.  

Inputs: global variables (Strength, Dexterity, Combat, HP, Attack Bonus, screen flag, monster existence flag, screen flag, AC Bonus)  
Local variables: damage

In [20]:
def defend():
    global ac_modifier, hp, promt1, promt2, player_exists, monster_exists, strg, dext, \
            strg, comb, combat_mode, screen_flag, attack_bonus, ac_bonus, kills
    
    ac_modifier += 3
    damage = monster_attack()
    hp -= damage 
    promt1 = "Monster dealed: " + str(damage) + " damage."
    if hp < 1:
        promt1 = ""
        promt2 = ""
        kills = 0
        medkits = 0
        fill_background()
        canvas.fill_style = "red"
        canvas.fill_text("YOU LOST!", 100, 100)
        sleep(4)
        monster_exists = False
        player_exists == False
        combat_mode = False
        screen_flag = "player_creation"
        strg = 1 
        dext = 1
        comb = 1
        hp = 10
        attack_modifier = 0
        ac_modifier = 0
        attack_bonus = 0
        ac_bonus =0 
        player_creation()
        return
    ac_modifier = 0
    game()
    return 


This function is resolving <b> monster attack turn</b>.   

It rolls D20 adds monster AC modifier, than checks it against player AC Bonus + AC Modifier.  
Than updates player HP and check if player is alive.  
If player dies game starts from begining.  

Inputs: global variables (AC Modifier, AC Bonus, monster Attack Bonus, Monster Damage)  


Returns: Damage dealt by monster

In [14]:
def monster_attack():
    global m_attack, m_damage, ac_bonus, ac_modifier
    
    if randint(1,20) + m_attack > ac_bonus + ac_modifier:
        return m_damage
    else:
        return 0

This function is resolving <b> Attack</b> button.   

It resolves two types of attack player can do, than calls monster attack function to resolve monster turn.  
Than updates player and Monster HP and check if player or monster is alive.  
If player dies game starts from begining.  
If monster dies <b>kills</b> counter is increased by 1. New monster is generated.

Inputs:  type of attack
global variables (Strength, Dexterity, Combat, HP, Attack Bonus, screen flag, monster existence flag, screen flag, AC Bonus)  


In [16]:
def attack(type_of_attack):
    global m_ac, m_hp, strg, dext,comb, hp, promt1, promt2, medkits, \
    player_exists, monster_exists, combat_mode, kills, ac_modifier, attack_bonus, ac_bonus
    
    if type_of_attack == "short":
        #Resolve short Attack
        result = randint(1,20) + attack_bonus 
        
        if result > m_ac:
            damage = randint(1,3) + strg // 2
            promt1 = "You dealed: "+str(damage)+" damage."
            m_hp -= damage
            if m_hp < 1:
                #Monster dies
                kills += 1
                monster_exists = False
                combat_mode = False
                promt1 = ""
                promt2 = ""
                game()
                return
        else: 
            promt1 ="You rolled: "+str(result)+" and missed."
            
        
        
        m_damage = monster_attack()
        ac_modifier = 0
        print("monster damage",m_damage)
        hp -= m_damage
        promt2 = "Monster dealed: "+str(m_damage)+" damage."
        if hp < 1:
            #Player dies
            promt1 = ""
            promt2 = ""
            kills = 0
            medkits = 0
            fill_background()
            canvas.fill_style = "red"
            canvas.fill_text("YOU LOST!",100,100)
            sleep(4)
            monster_exists = False
            player_exists == False
            combat_mode = False
            screen_flag = "player_creation"
            strg = 1 
            dext = 1
            comb = 1
            hp = 10
            attack_modifier = 0
            ac_modifier = 0
            attack_bonus = 0
            ac_bonus =0 
            player_creation()
            return
    else:
        #Resolve long attack +4
        result = randint(1,20) + attack_bonus + 4
        ac_modifier -= 4
        
        if result > m_ac:
            damage = randint(1,6) + strg // 2
            promt1 = "You dealed: "+str(damage)+" damage."
            m_hp -= damage
            if m_hp < 1:
                #Monster dies
                kills += 1
                monster_exists = False
                combat_mode = False
                promt1 = ""
                promt2 = ""
                ac_modifier = 0
                game()
                return
        else: 
            promt1 ="You rolled: "+str(result)+" and missed."
            
        
        
        m_damage = monster_attack() + monster_attack()
        ac_modifier = 0
        print("monster damage", m_damage)
        hp -= m_damage
        promt2 = "Monster dealed: "+str(m_damage)+" damage."
        if hp < 1:
            #Player dies
            promt1 = ""
            promt2 = ""
            kills = 0
            medkits = 0
            fill_background()
            canvas.fill_style = "red"
            canvas.fill_text("YOU LOST!",100,100)
            sleep(4)
            monster_exists = False
            player_exists == False
            combat_mode = False
            screen_flag = "player_creation"
            strg = 1 
            dext = 1
            comb = 1
            hp = 10
            attack_modifier = 0
            ac_modifier = 0
            attack_bonus = 0
            ac_bonus =0 
            player_creation()
            return
    game()
    return


That function <b>generates monster stats depending on ammount of kills</b>. The more the kills the more difficult monster will be generated.

Inputs: global variable (kills)  
Outputs: monster hp, monster attack, monster ac, monster damage

In [17]:
def generate_monster_stats():
    #returns: m_hp, m_attack, m_ac, m_damage
    global kills
    
    if kills < 5:
        return randint(4, 10), 0, randint(6, 10), randint(1, 2)
    elif kills >= 5 and kills < 20:
        return randint(10, 15), randint(1, 2), randint(10, 12), randint(3, 5)
    else:
        return randint(15, 18), randint(1, 2), randint(12, 15), randint(3, 5)

That is main function which is handling events and calling corresponding functions.
Updates the screen. And redraws it.


Inputs: global variable (screen_flag,strg, dext, pl_x, pl_y, m_x, m_y, med_x, med_y, monster_exists, medkit_exists, player_exists,up_btn, r_btn, l_btn, dn_btn, pu_btn, combat_mode, hp, strg, dext, comb, medkits, combat_mode, m_hp, m_attack, m_ac, m_damage,promt1, promt2, kills)


In [18]:
def game():
    global screen_flag,strg, dext, pl_x, pl_y, m_x, m_y, med_x, med_y,\
    monster_exists, medkit_exists, player_exists,up_btn, r_btn, l_btn, dn_btn, pu_btn, \
    combat_mode, hp, strg, dext, comb, medkits, combat_mode, m_hp, m_attack, m_ac, m_damage,promt1, promt2, \
    kills
    
    if screen_flag == "player_creation":
        player_creation()
        return
    
    fill_background()
    draw_game_screen()
    if player_exists == False:
        pl_x, pl_y = generate_coordinates(130,130)
        player_exists = True
    if monster_exists == False:
        m_x, m_y = generate_coordinates(pl_x, pl_y)
        m_hp, m_attack, m_ac, m_damage = generate_monster_stats()
        monster_exists = True
    if medkit_exists == False:
        med_x, med_y = generate_coordinates(pl_x, pl_y)
        medkit_exists = True
    draw_player(pl_x, pl_y, "#50F40C")
    draw_player(m_x, m_y, "red")
    draw_medkit(med_x, med_y)
    combat_promt()
           
    return


### 3. Lets Test the Game!

Initializing global variables.  
Calling player creation.
Starting Game.

In [19]:
promt1 =""
promt2 =""
strg = 1 
dext = 1
comb = 1
attack_modifier = 0
ac_modifier = 0
attack_bonus = 0
ac_bonus =0 



screen_width = 700
screen_hight = 500

battle_screen_x_min = 130
battle_screen_x_max = battle_screen_x_min + screen_width - 260
battle_screen_y_min = 130
battle_screen_y_max = battle_screen_y_min + screen_hight - 260

up_btn = True 
r_btn = True
l_btn = True
dn_btn = True
pu_btn = False

monster_exists = False 
medkit_exists = False
player_exists = False
medkits = 0
hp = 10
kills = 0

game_mode = "easy"
combat_mode = False


canvas = Canvas(width = screen_width, hight = screen_hight)

fill_background()
player_creation()
display(canvas)
canvas.on_mouse_down(handle_mouse_down)

Canvas()